In [51]:
import numpy as np
import pandas as pd
from google.colab import files

In [52]:
df = pd.read_csv('https://raw.githubusercontent.com/Nachoxt17/Real-Estate-Price-Estimator-for-Tokyo/main/data/tokyo_real_estate_data.csv')

<hr>

# Data Preprocessing

## 1 - Look at data

Let’s see the dimensions of the dataframe.

In [53]:
df.shape
df.head()

,Type,Region,MunicipalityCode,Prefecture,Municipality,DistrictName,NearestStation,TimeToNearestStation,MinTimeToNearestStation,MaxTimeToNearestStation,...,Classification,Breadth,CityPlanning,CoverageRatio,FloorAreaRatio,Period,Year,Quarter,Renovation,Remarks
0,"Pre-owned Condominiums, etc.",NaN,13101,Tokyo,Chiyoda Ward,Iidabashi,Iidabashi,4,4.0,4.0,...,NaN,NaN,Commercial Zone,80.0,500.0,3rd quarter 2019,2019,3,Not yet,NaN
1,"Pre-owned Condominiums, etc.",NaN,13101,Tokyo,Chiyoda Ward,Iidabashi,Iidabashi,4,4.0,4.0,...,NaN,NaN,Commercial Zone,80.0,500.0,3rd quarter 2019,2019,3,Done,NaN
2,Residential Land(Land and Building),Commercial Area,13101,Tokyo,Chiyoda Ward,Iidabashi,Iidabashi,2,2.0,2.0,...,Ward Road,4.0,Commercial Zone,80.0,500.0,2nd quarter 2019,2019,2,NaN,NaN
3,Residential Land(Land and Building),Commercial Area,13101,Tokyo,Chiyoda Ward,Iidabashi,Iidabashi,1,1.0,1.0,...,Tokyo Metropolitan Road,27.0,Commercial Zone,80.0,700.0,2nd quarter 2019,2019,2,NaN,NaN
4,"Pre-owned Condominiums, etc.",NaN,13101,Tokyo,Chiyoda Ward,Iidabashi,Iidabashi,4,4.0,4.0,...,NaN,NaN,Commercial Zone,80.0,500.0,2nd quarter 2019,2019,2,Done,NaN


<hr>

## 2 - Remove Unnecessary Columns

Among all columns, we don’t need some of them. So we drop them from dimension.

In [54]:
df = df.drop(['MunicipalityCode', 'Prefecture', 'Region', 'DistrictName', 'NearestStation','MinTimeToNearestStation', 'MaxTimeToNearestStation',
              'AreaIsGreaterFlag', 'FrontageIsGreaterFlag', 'TotalFloorArea',
              'TotalFloorAreaIsGreaterFlag', 'PrewarBuilding', 'Purpose', 'Classification', 'Breadth',
              'CityPlanning', 'CoverageRatio', 'FloorAreaRatio', 'Period', 'Year','Quarter','Remarks'] , axis = 1)

In [55]:
df.head()

,Type,Municipality,TimeToNearestStation,TradePrice,FloorPlan,Area,LandShape,Frontage,BuildingYear,Structure,Use,Direction,Renovation
0,"Pre-owned Condominiums, etc.",Chiyoda Ward,4,40000000,1LDK,30,NaN,NaN,2013.0,RC,House,NaN,Not yet
1,"Pre-owned Condominiums, etc.",Chiyoda Ward,4,130000000,3LDK,80,NaN,NaN,2007.0,RC,NaN,NaN,Done
2,Residential Land(Land and Building),Chiyoda Ward,2,400000000,NaN,110,Trapezoidal Shaped,9.0,2018.0,S,"Office, Shop",Northwest,NaN
3,Residential Land(Land and Building),Chiyoda Ward,1,180000000,NaN,50,Rectangular Shaped,5.2,1976.0,RC,"Housing Complex, Office",Southwest,NaN
4,"Pre-owned Condominiums, etc.",Chiyoda Ward,4,100000000,2LDK,65,NaN,NaN,2007.0,RC,House,NaN,Done


<hr>

## 3 - Change Column’s Name

I change the name of the column based on the following:
* TimeToNearestStation: NearStation
* TradePrice: Price

In [56]:
col_name = ['Type', 'Municipality','NearStation',
       'Price', 'FloorPlan', 'Area', 'LandShape', 'Frontage',
       'BuildingYear', 'Structure', 'Use', 'Direction', 'Renovation']

df.columns = col_name

print(df.columns)

Index(['Type', 'Municipality', 'NearStation', 'Price', 'FloorPlan', 'Area',
       'LandShape', 'Frontage', 'BuildingYear', 'Structure', 'Use',
       'Direction', 'Renovation'],
      dtype='object')


<hr>

## 4 - Handle Missing Values

I create a for loop to iterate among all columns to realize whether they have null values or not. I’m looking for the number of null values in every single column as well as the percentage of null values.

In [57]:
for col in df.columns:
    number_null = df.loc[: , col].isnull().sum()
    perc_null = (number_null / df.shape[0]) * 100
    print('{} - {} - %{}'.format(col, number_null, round(perc_null,3)))

Type - 0 - %0.0
Municipality - 0 - %0.0
NearStation - 137 - %0.685
Price - 0 - %0.0
FloorPlan - 3153 - %15.765
Area - 0 - %0.0
LandShape - 16848 - %84.24
Frontage - 17053 - %85.265
BuildingYear - 710 - %3.55
Structure - 488 - %2.44
Use - 2208 - %11.04
Direction - 16847 - %84.235
Renovation - 4917 - %24.585


The result shows we must have a different approach to handling null values. Since I have a large dataset:
<br><br>
Categorical:
* less than 5%, I drop the rows.
* between 5% and 30%, I impute with mode.
* More than 30%, create a new label as “Other.”
<br><br>

Numerical:
* between 0% and 30%, I impute with mean or median.
* More than 30%, I drop the rows.

<br>
However, the best way is consulting with expert domain.

<hr>

Let’s begin with categorical variables. First, I impute “Renovation”, “FloorPlan” and "Use" with mode.

In [58]:
mode_Renovation = df['Renovation'].mode()[0]
mode_Use = df['Use'].mode()[0]
mode_FloorPlan = df['FloorPlan'].mode()[0]

print(mode_Renovation)
print(mode_Use)
print(mode_FloorPlan)

Not yet
House
1K


In [59]:
df['Renovation'] = df['Renovation'].fillna('Not yet')
df['Use'] = df['Use'].fillna('House')
df['FloorPlan'] = df['FloorPlan'].fillna('1K')

Then, I created a new label for missing values in “LandShape”, “Direction”.

In [60]:
df['LandShape'] = df['LandShape'].fillna('Other LandShape')
df['Direction'] = df['Direction'].fillna('Other Direction')

Finally, I drop “structure” null value.

In [61]:
df = df.dropna(subset = ['Structure'])

<hr>

And now, I work on numerical variables. First, I impute “NearStation” and “year. But before doing this, I must make sure about distribution shape of these columns to see whether they are right-skewed or left-skewed. It can be helpful when I want to decide choosing mean or median for imputing. Also, I should check the data type of theses variables to be sure about their numerical type.

In [62]:
print(df['NearStation'].dtypes)
print(df['BuildingYear'].dtypes)

object
float64


"NearStation" is object and it means there are some values in this column that are not number. First, we should handle this issue. So, I check the unique values:

In [63]:
df['NearStation'].unique()

array(['4', '2', '1', nan, '3', '5', '6', '0', '7', '11', '10', '8', '9',
       '12', '30-60minutes', '13', '14', '16', '15', '19', '18', '21',
       '2H-', '17'], dtype=object)

As we can see, there are some values that are not number, and they are range. So, I ignore those kinds of values from model.

In [64]:
df['NearStation'] = pd.to_numeric(df['NearStation'], errors='coerce')

In [65]:
df['NearStation'].unique()

array([ 4.,  2.,  1., nan,  3.,  5.,  6.,  0.,  7., 11., 10.,  8.,  9.,
       12., 13., 14., 16., 15., 19., 18., 21., 17.])

Now we can see its distribution.

In [66]:
print('Skewness :' , round(df['NearStation'].skew() ,3))

mean_NearStation = df['NearStation'].mean()
median_NearStation = df['NearStation'].median()

if mean_NearStation > median_NearStation:
    print('Mean is bigger than Median. Left Skewed. Median for imputing')
else:
    print('Mean is smaller than Median. Right Skewed. Mean for imputing')

Skewness : 1.103
Mean is bigger than Median. Left Skewed. Median for imputing


The result shows that I should choose median for imputing.

In [67]:
df['NearStation'] = df['NearStation'].fillna(median_NearStation).round(0)

In the next, I drop null values for “Frontage”.

In [68]:
df = df.dropna(subset = ['Frontage'])

run the same process for "BulidingYear".

In [69]:
print('Skewness :' , round(df['BuildingYear'].skew() ,3))

mean_BuildingYear = df['BuildingYear'].mean()
median_BuildingYear = df['BuildingYear'].median()

if mean_BuildingYear > median_BuildingYear:
    print('Mean is bigger than Median. Left Skewed. Median for imputing')
else:
    print('Mean is smaller than Median. Right Skewed. Mean for imputing')

Skewness : 0.098
Mean is smaller than Median. Right Skewed. Mean for imputing


The result shows that I should choose mean for imputing.

In [70]:
df['BuildingYear'] = df['BuildingYear'].fillna(mean_BuildingYear).round(0)

<ipython-input-70-0280c4d9c74d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['BuildingYear'] = df['BuildingYear'].fillna(mean_BuildingYear).round(0)


Finally, we check the null values for dataset again.

In [71]:
for col in df.columns:
    number_null = df.loc[: , col].isnull().sum()
    perc_null = (number_null / df.shape[0]) * 100
    print('{} - {} - %{}'.format(col, number_null, round(perc_null,3)))

Type - 0 - %0.0
Municipality - 0 - %0.0
NearStation - 0 - %0.0
Price - 0 - %0.0
FloorPlan - 0 - %0.0
Area - 0 - %0.0
LandShape - 0 - %0.0
Frontage - 0 - %0.0
BuildingYear - 0 - %0.0
Structure - 0 - %0.0
Use - 0 - %0.0
Direction - 0 - %0.0
Renovation - 0 - %0.0


<hr>

## 5 - Handle Duplicate Rows

Now we should handle duplicate rows. Since all values might be same, we just we need to check whether there are two rows that all values in all columns are the same or not.

In [72]:
duplicate_rows = df.duplicated()

if duplicate_rows.any():
    print("The DataFrame has duplicate rows.")
else:
    print("The DataFrame does not have duplicate rows.")

The DataFrame has duplicate rows.


The result shows we have duplicated rows. Now, let’s see the duplicated rows:

In [73]:
duplicate_rows = df[df.duplicated(keep=False)]

duplicate_rows = pd.DataFrame(duplicate_rows)
duplicate_rows

,Type,Municipality,NearStation,Price,FloorPlan,Area,LandShape,Frontage,BuildingYear,Structure,Use,Direction,Renovation
4986,Residential Land(Land and Building),Chuo Ward,2.0,230000000,1K,80,Rectangular Shaped,7.0,1964.0,SRC,Office,Northeast,Not yet
4994,Residential Land(Land and Building),Chuo Ward,2.0,230000000,1K,80,Rectangular Shaped,7.0,1964.0,SRC,Office,Northeast,Not yet
9353,Residential Land(Land and Building),Minato Ward,1.0,360000000,1K,160,Rectangular Shaped,8.9,1991.0,RC,"Housing Complex, Office, Shop",North,Not yet
9354,Residential Land(Land and Building),Minato Ward,1.0,360000000,1K,160,Rectangular Shaped,8.9,1991.0,RC,"Housing Complex, Office, Shop",North,Not yet
14686,Residential Land(Land and Building),Shinjuku Ward,5.0,59000000,1K,50,Rectangular Shaped,7.2,2015.0,W,House,North,Not yet
14690,Residential Land(Land and Building),Shinjuku Ward,5.0,59000000,1K,50,Rectangular Shaped,7.2,2015.0,W,House,North,Not yet
18342,Residential Land(Land and Building),Shinjuku Ward,4.0,400000000,1K,145,Semi-rectangular Shaped,9.5,2014.0,RC,Housing Complex,North,Not yet
18343,Residential Land(Land and Building),Shinjuku Ward,4.0,400000000,1K,145,Semi-rectangular Shaped,9.5,2014.0,RC,Housing Complex,North,Not yet


In [74]:
print("before: ", df.shape)
df = df.drop_duplicates(keep='first')
print("after: ", df.shape)

before:  (2765, 13)
after:  (2761, 13)


Then, we can check duplicate rows again.

In [75]:
duplicate_rows = df.duplicated()

if duplicate_rows.any():
    print("The DataFrame has duplicate rows.")
else:
    print("The DataFrame does not have duplicate rows.")

The DataFrame does not have duplicate rows.


<hr>

## 6 - Handle Number Variables

First of all, I declare all number variables. The number variables are: “Area”, “Frontage”, “NearStation”, “BuildingYear”, “Price”.

In [76]:
num_list = ['Area', 'Frontage', 'NearStation', 'BuildingYear', 'Price']

I must make sure about the data type of the number variable. Just because the column shows numbers, it doesn't mean that they are numbers.Thus, with regular expression I should clean them.

In [77]:
def non_numeric(x):
    non_numeric_df = pd.DataFrame(df[df[x].astype(str).str.contains('[^\d\.]+')])
    return non_numeric_df

In [78]:
non_numeric('Area')

,Type,Municipality,NearStation,Price,FloorPlan,Area,LandShape,Frontage,BuildingYear,Structure,Use,Direction,Renovation


In [79]:
non_numeric('Frontage')

,Type,Municipality,NearStation,Price,FloorPlan,Area,LandShape,Frontage,BuildingYear,Structure,Use,Direction,Renovation


In [80]:
non_numeric('NearStation')

,Type,Municipality,NearStation,Price,FloorPlan,Area,LandShape,Frontage,BuildingYear,Structure,Use,Direction,Renovation


In [81]:
non_numeric('BuildingYear')

,Type,Municipality,NearStation,Price,FloorPlan,Area,LandShape,Frontage,BuildingYear,Structure,Use,Direction,Renovation


In [82]:
non_numeric('Price')

,Type,Municipality,NearStation,Price,FloorPlan,Area,LandShape,Frontage,BuildingYear,Structure,Use,Direction,Renovation


The results show fortunately in those variables, we just have number nothing else. However, we can see the sum of those variables for double check.

In [83]:
for i in range(len(num_list)):
    var_sum = df.loc[: , num_list[i]].sum()
    print(num_list[i] , var_sum)

Area 470260
Frontage 26427.699999999997
NearStation 12597.0
BuildingYear 5488313.0
Price 1424412020000


<hr>

## 7 - Handle Categorical Variables

First of all, I declare all categorical variables. These variables are: “Province”, “City”, “Type”, “LandShape”, “Structure”, “FloorPlan”, ‘Direction”, “Use”, “Renovation”.

In [84]:
cat_list = ['Type', 'Municipality', 'FloorPlan', 'LandShape', 'Structure', 'Direction', 'Use', 'Renovation']

Then, we must make sure about the possible range for each of them. They must be the same with data documentation.

> ### Type

In [85]:
 print(df['Type'].unique())

['Residential Land(Land and Building)']


> ### Municipality

In [86]:
 print(df['Municipality'].unique())

['Chiyoda Ward' 'Chuo Ward' 'Minato Ward' 'Shinjuku Ward' 'Bunkyo Ward']


In [87]:
 print(df['Municipality'].value_counts())

Municipality
Shinjuku Ward    1015
Minato Ward       660
Chuo Ward         581
Chiyoda Ward      434
Bunkyo Ward        71
Name: count, dtype: int64


> ### FloorPlan

In [88]:
print(df['FloorPlan'].unique())

['1K']


> ### LandShape

In [89]:
print(df['LandShape'].unique())

['Trapezoidal Shaped' 'Rectangular Shaped' 'Semi-rectangular Shaped'
 'Semi-trapezoidal Shaped' 'Irregular Shaped' 'Semi-shaped'
 'Semi-square Shaped' 'Flag-shaped etc.' 'Square Shaped' 'Other LandShape']


In [90]:
 print(df['LandShape'].value_counts())

LandShape
Semi-rectangular Shaped    1014
Rectangular Shaped          731
Irregular Shaped            423
Semi-trapezoidal Shaped     197
Semi-square Shaped          157
Semi-shaped                 111
Trapezoidal Shaped           78
Flag-shaped etc.             28
Square Shaped                21
Other LandShape               1
Name: count, dtype: int64


> ### Structure

In [91]:
print(df['Structure'].unique())

['S' 'RC' 'SRC' 'W' 'LS' 'S, LS' 'B' 'SRC, RC' 'S, W' 'RC, W' 'RC, S'
 'W, B']


All of them are correct and based on data documentation. However, some of them have two or more types of structure, and because of reduced-dimensionality, I just consider the first one.

In [92]:
df['Structure'] = df['Structure'].apply(lambda x: x.split(',')[0])

In [93]:
 print(df['Structure'].value_counts())

Structure
RC     1014
W       795
S       551
SRC     348
LS       50
B         3
Name: count, dtype: int64


> ### Direction

In [94]:
print(df['Direction'].unique())

['Northwest' 'Southwest' 'South' 'Northeast' 'East' 'Southeast' 'West'
 'North' 'No facing road']


In [95]:
 print(df['Direction'].value_counts())

Direction
Northwest         382
Southwest         373
Southeast         371
West              344
Northeast         335
South             329
East              307
North             304
No facing road     16
Name: count, dtype: int64


> ### Use

In [96]:
print(df['Use'].unique())

['Office, Shop' 'Housing Complex, Office' 'House, Office, Workshop'
 'House, Office, Shop' 'Other' 'Office, Warehouse' 'Parking Lot'
 'Office, Parking Lot' 'Housing Complex, Office, Shop' 'House' 'Office'
 'Housing Complex' 'Housing Complex, Shop' 'House, Shop' 'Warehouse, Shop'
 'Housing Complex, Office, Warehouse, Shop' 'House, Workshop, Shop'
 'Warehouse' 'House, Office' 'Office, Warehouse, Parking Lot'
 'House, Factory, Office, Shop' 'House, Office, Other' 'Shop'
 'House, Office, Warehouse, Shop' 'Office, Other'
 'Office, Warehouse, Shop' 'House, Warehouse, Shop, Other' 'Workshop'
 'House, Office, Parking Lot' 'Office, Workshop, Warehouse' 'Factory'
 'House, Warehouse' 'House, Housing Complex, Factory, Warehouse'
 'Office, Parking Lot, Shop' 'House, Factory, Office'
 'Housing Complex, Office, Warehouse, Parking Lot'
 'House, Office, Warehouse' 'House, Workshop' 'House, Office, Shop, Other'
 'House, Parking Lot' 'House, Parking Lot, Shop'
 'Housing Complex, Parking Lot' 'Housing Com

All of them are correct and based on data documentation. However, some of them have two or more types of use, and because of reduced-dimensionality, I just consider the first one.

In [97]:
df['Use'] = df['Use'].apply(lambda x: x.split(',')[0])

In [98]:
print(df['Use'].value_counts())

Use
House              1300
Office              705
Housing Complex     559
Shop                117
Other                43
Warehouse            18
Factory               9
Parking Lot           6
Workshop              4
Name: count, dtype: int64


> ### Renovation

In [99]:
print(df['Renovation'].unique())

['Not yet']


<hr>

# Check Point

In [100]:
# Save Processed DataFrame to C.S.V.:
df.to_csv('df_preprocessed.csv', index=False)

# Download C.S.V. to your local computer:
files.download('df_preprocessed.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>